In [1]:
#import libraries
import sqlite3 as db
import pandas as pd

In [2]:
#connect to database or create if doesn't exist
conn = db.connect('my_database.db')

In order to interact with our database, we now have to create a cursor. This is what we will use to issue commands that will allow us to query or modify our database.



In [3]:
#create cursor
c = conn.cursor()

Create table

In [4]:
#create table called 'employees'
c.execute("CREATE TABLE employees (empid INTEGER PRIMARY KEY, firstname NVARCHAR(20), lastname NVARCHAR(20))")

In [5]:
#check that table was successfully created in database
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('employees',)]



Insert data into table
Option 1 - insert a few rows

In [6]:
c.execute("INSERT INTO employees VALUES (12985,'Michael','Scott')")
c.execute("INSERT INTO employees VALUES (12986,'Dwight', 'Schrute')")

In [7]:
#check data in table
c.execute("SELECT * FROM employees;")
print(c.fetchall())

[(12985, 'Michael', 'Scott'), (12986, 'Dwight', 'Schrute')]


Option 2 - insert many rows

In [8]:
new_employees = [(12987, 'Jim', 'Halpert'),
 (12988, 'Pam', 'Beesly'),
 (12989, 'Andy', 'Bernard'),
 (12990, 'Kevin', 'Malone'),
 (12991, 'Toby', 'Flenderson'),
 (12992, 'Angela', 'Martin'),
 (12993, 'Stanley', 'Hudson')]

In [9]:
c.executemany('INSERT INTO employees VALUES (?, ?, ?)', new_employees)

In [10]:
#check data in table
c.execute("SELECT * FROM employees;")
print(c.fetchall())

[(12985, 'Michael', 'Scott'), (12986, 'Dwight', 'Schrute'), (12987, 'Jim', 'Halpert'), (12988, 'Pam', 'Beesly'), (12989, 'Andy', 'Bernard'), (12990, 'Kevin', 'Malone'), (12991, 'Toby', 'Flenderson'), (12992, 'Angela', 'Martin'), (12993, 'Stanley', 'Hudson')]


Notice that we’ve replaced our insert statement from earlier with (?, ?, ?). The question marks here represent placeholders for the three items that we are inputting into the table.

Committing changes

We’ve made a couple changes to our database, but they are not saved to the database just yet! These are just temporary modifications, so we have to make sure to commit those changes with the following command.

In [11]:
conn.commit()

Close connections

In [12]:
c.close()
conn.close()

Drop table if you need to

In [13]:
#c.execute("DROP TABLE IF EXISTS employees")

SQLite via Pandas

One cool thing you can do is use both SQLite and Pandas. Pandas has a read_sql_query method that will allow you to return the data as a Pandas dataframe. From there, you can more easily manipulate the data in Pandas. Personally, I prefer bringing the data inside Pandas to work with.

We again have to first establish a connection to our database. Then we can use pd.read_sql_queryand save the output as a dataframe calleddf_employees.

In [14]:
#Establish connection
con = db.connect('my_database.db')

In [15]:
#Perform SQL query and return pandas data frame
df_employees = pd.read_sql_query ('select * from employees', con)
df_employees

,empid,firstname,lastname
0,12985,Michael,Scott
1,12986,Dwight,Schrute
2,12987,Jim,Halpert
3,12988,Pam,Beesly
4,12989,Andy,Bernard
5,12990,Kevin,Malone
6,12991,Toby,Flenderson
7,12992,Angela,Martin
8,12993,Stanley,Hudson


In [16]:
#remove Stanley from employee table
df_new = df_employees[:-1]
df_new

,empid,firstname,lastname
0,12985,Michael,Scott
1,12986,Dwight,Schrute
2,12987,Jim,Halpert
3,12988,Pam,Beesly
4,12989,Andy,Bernard
5,12990,Kevin,Malone
6,12991,Toby,Flenderson
7,12992,Angela,Martin


In [17]:
#Write the new dataframe to SQLite and replace employee table
df_new.to_sql("employees", con, if_exists="replace")

In [18]:
#Check table in SQLite
pd.read_sql_query ('select * from employees', con)

,index,empid,firstname,lastname
0,0,12985,Michael,Scott
1,1,12986,Dwight,Schrute
2,2,12987,Jim,Halpert
3,3,12988,Pam,Beesly
4,4,12989,Andy,Bernard
5,5,12990,Kevin,Malone
6,6,12991,Toby,Flenderson
7,7,12992,Angela,Martin


In [20]:

#Close connection

con.close()